In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile
import os

zip_file_path = "/content/drive/MyDrive/cropped_rois.zip"

extract_path = "/content"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Successfully unzipped '{zip_file_path}' to '{extract_path}'")


Successfully unzipped '/content/drive/MyDrive/cropped_rois.zip' to '/content'


In [4]:
import os
import cv2
import numpy as np
from glob import glob
from skimage.feature import hog
from sklearn.preprocessing import Binarizer
from IPython.display import display, Image

In [7]:
def extract_pqhog(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, _ = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(1, 1),
                      orientations=9, visualize=True, block_norm='L2-Hys')
    binarized = Binarizer(threshold=0).fit_transform(features.reshape(1, -1)).flatten()
    return binarized

In [5]:
def generate_pqhog_descriptors(roi_folder="cropped_rois/test", save_folder="pqhog_descriptors/test"):
    os.makedirs(save_folder, exist_ok=True)
    roi_paths = glob(os.path.join(roi_folder, "*.jpg"))

    for path in roi_paths:
        image = cv2.imread(path)
        if image is None:
            continue
        descriptor = extract_pqhog(image)
        name = os.path.basename(path).replace(".jpg", "_pqhog.npy")
        np.save(os.path.join(save_folder, name), descriptor)

    print(f"✅ Saved {len(roi_paths)} descriptors to {save_folder}")


In [14]:
generate_pqhog_descriptors("/content/content/cropped_rois/valid", "pqhog_descriptors/valid")


In [9]:
import pickle
from collections import defaultdict

def build_ili_index(descriptor_folder="pqhog_descriptors/test", save_path="ili_index.pkl"):
    ili = defaultdict(list)
    for file in glob(os.path.join(descriptor_folder, "*.npy")):
        desc = np.load(file)
        key = tuple(desc)  # Binary descriptor as key
        ili[key].append(file)
    with open(save_path, "wb") as f:
        pickle.dump(ili, f)
    print(f"✅ ILI index saved to {save_path} with {len(ili)} unique entries")


In [ ]:
!mkdir templates
!cp cropped_rois/test/test_example1.jpg templates/
!cp cropped_rois/test/test_example2.jpg templates/


In [12]:
# prompt: zip folder

import zipfile
import os

def zip_folder(folder_path, zip_file_name):
    """Zips a folder and its contents.

    Args:
        folder_path: The path to the folder to be zipped.
        zip_file_name: The name of the zip file to create (including .zip extension).
    """
    try:
        with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, folder_path)  # Correct relative path
                    zipf.write(file_path, arcname=arcname)
        print(f"Successfully zipped '{folder_path}' to '{zip_file_name}'")
    except FileNotFoundError:
        print(f"Error: Folder '{folder_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:  Replace with your desired folder and zip file name
zip_folder("pqhog_descriptors", "pqhog_descriptors.zip")


Successfully zipped 'pqhog_descriptors' to 'pqhog_descriptors.zip'


In [13]:
!cp pqhog_descriptors.zip /content/drive/MyDrive/